In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 

from fnope.utils.misc import get_output_dir


In [ ]:
#### Load data to be plotted from csv ####
out_dir = get_output_dir()

path = out_dir / "linear_gaussian_experiment/summary.csv"
data = pd.read_csv(path)
methods = data['method'].unique()
n_sim = data['nsim'].unique()

# Calculate mean and SE for each method and each number of simulations
swd_results = {}
swd_mean = np.zeros((methods.shape[0], n_sim.shape[0]))
swd_SE = np.zeros((methods.shape[0], n_sim.shape[0]))

for ii, method in enumerate(methods):
    swd_results[method] = {}
    for kk, nsim in enumerate(n_sim):
        swd_results[method][nsim] = []
        temp_swds = data[(data['method'] == method) & (data['nsim'] == nsim)]['swds']
        for ll in range(temp_swds.shape[0]):
            clean_string = temp_swds.iloc[ll].strip("[]")
            swd_results[method][nsim].extend(list(map(float, clean_string.split())))
        swd_mean[ii, kk] = np.mean(np.array(swd_results[method][nsim]))
        swd_SE[ii, kk] = np.std(np.array(swd_results[method][nsim]))/np.sqrt(len(swd_results[method][nsim]))



In [ ]:
# plotting with matplotlibrc file 

with plt.rc_context(fname='./matplotlibrc'):
    
    fig, ax = plt.subplots(1, 1, figsize=(1.9, 1.4))

    methods_names = ['FNOPE', 'FNOPE (fix)',  'NPE (spectral)', 'FMPE (raw)', 'FMPE (spectral)']
    colors = ['#9b2226', '#CA6702', '#023e8a', '#00b4d8', '#0077b6']

    for mm in range(methods.shape[0]):

        ax.errorbar(n_sim,
                    swd_mean[mm,:],
                    yerr=swd_SE[mm,:],
                    fmt='o',
                    linestyle='-',
                    color=colors.pop(0),
                    label=methods_names[mm])
        
    ax.set_xscale("log")
    ax.set_xlabel('# simulations')
    ax.set_ylabel('SWD')
    ax.set_xticks(n_sim)
    ax.set_yticks([0, 1.0])
    ax.minorticks_off()
    #ax.legend(loc='upper left', bbox_to_anchor=(1, 0.7))
    # Create custom legend order
    custom_order = [0, 1, 2, 4, 3]  # New order of methods_names

    # Get handles and labels
    handles, labels = ax.get_legend_handles_labels()

    # Reorder handles and labels based on custom order
    handles = [handles[i] for i in custom_order]
    labels = [labels[i] for i in custom_order]
    # Create the legend with the new order
    ax.legend(handles=handles,
                labels=labels,
                loc='upper left',
                bbox_to_anchor=(1.1, 1.1),
                handlelength=1.5,
                fontsize=10,
                ncol=1)

    
    ax.spines['left'].set_position(('outward', 5))     # Move y-axis slightly left
    ax.spines['bottom'].set_position(('outward', 5))

    # plt.savefig('linear_gaussian_plots/results_linearGaussian.pdf', format='pdf', bbox_inches='tight')
    plt.show()